# SQL/SQLite com Pandas

Este notebook serve para gerenciar e manipular o DB SQLite com todos os datasets usados em aula.

## Índice

1. [Importação de um CSV para um DB com Pandas](#python-import)
2. [Importação de um CSV para um DB com a linha de comando (mais rápido, permite arquivos maiores, mas com menos controle)](#shell-import)
3. [Atualização da tabela-catálogo chamada **00datasets**](#00datasets)
4. [Faça queries no banco de dados pronto](#query)

<a id="python-import"></a>
## 1. Como importar um CSV no DB, com Pandas

In [ ]:
import pandas as pd
import sqlite3


dbfile = "datasets.db"
csv = "air-passengers.zip"
nova_tabela = "kaggle_air_passengers"

db = sqlite3.connect(dbfile)

df = pd.read_csv(csv)

df.head(30)

Insira aqui algumas manipulações que queira fazer no dataframe antes de mandá-lo para o DB...

In [ ]:
# ...

# df.drop('rowid', axis=1, inplace=True)
# df

Determina o tipo de cada coluna baseado no `dtype` da `Series` da coluna.

In [ ]:
sqlDataTypes={}

for c in df.columns:
    if df[c].dtype.kind == 'i':
        sqlDataTypes[c]='INTEGER'
    elif df[c].dtype.kind == 'f':
        sqlDataTypes[c]='REAL'
    else:
        sqlDataTypes[c]='TEXT'

sqlDataTypes

Agora escreve no DB

In [ ]:
df.to_sql(nova_tabela, index=False, if_exists='replace', dtype=sqlDataTypes, con=db)

Depois de inserir todos os dados e todas as tabelas, dê uma limpada (com o `vacuum`) e feche o banco

In [ ]:
db.cursor().execute("VACUUM")
db.commit()
db.close()

Este método cria uma tabela com o conteúdo do dataframe.

<a id="shell-import"></a>
## 2. Como importar um CSV no DB com a linha de comando do sistema operacional

O método anterior, com Pandas, é muito limpo porque permite manipulações. Mas está sujeito aos limites do Pandas e de sua RAM (memória do computador). O método Pandas não vai funcionar caso seu CSV seja gigatesco.

O método a seguir é mais simples e usa somente SQL do SQLite. É incrivelmente rápido e funciona com qualquer tamanho de arquivo CSV. Mas importa os dados (cria tabelas no DB) sem se importar muito com tipos de cada coluna (vai criar tudo como coluna TEXT). Qualquer otimização e transformação deve ser feita depois, com SQL.

Se você não tem o comando **sqlite3** (Mac e Linux têm), use o [SQLite Browser](https://sqlitebrowser.org/) (Windows, Mac e Linux) com os comandos `.mode` e `.import` abaixo. Ou com suas opções de menu.

<a id="00datasets"></a>
## 3. Não esqueça de atualizar a tabela **00datasets** !!!!!!!!!!!!11

A tabela **00datasets** contém metadados sobre:
* o nome da tabela/dataset
* de onde veio
* quem incluiu no DB
* aula e notebook onde é usado

In [ ]:
import pandas as pd
import sqlite3

dbfile = "datasets.db"
controle = "00datasets"

db = sqlite3.connect(dbfile)
datasets = pd.read_sql_query(f'select * from "{controle}" order by origin desc, usedOn desc', db)

datasets

In [ ]:
# Edite este valores com seu nome, URL de origem etc.
# Se não tiver com o que completar, deixe a linha em comentário para usar o que já estava no DB ou meter NULL.

criador = "EXEMPLO:Mané da Silva"
# origem = "EXEMPLO:http://algumlugar.com"
usado = "EXEMPLO:59 Módulo/03 Aula/notebook.ipynb;60 Módulo/02 Aula/notebook.ipynb"
nova_tabela = nova_tabela

# Pare de editar aqui. Código genérico abaixo...

linha=dict(datasets[datasets.name==nova_tabela])
linha['name']=nova_tabela

try:
    linha['createdBy']=criador
except NameError:
    pass

try:
    linha['origin']=origem
except NameError:
    pass

try:
    linha['usedOn']=usado
except NameError:
    pass


atualizador = """
    INSERT OR REPLACE INTO '00datasets'
    VALUES (:name,:createdBy,:origin,:usedOn)
"""

db.execute(atualizador,linha)
db.commit()
db.execute("VACUUM") # dá uma otimizadinha no DB
db.commit()
db.close()

<a id="query"></a>
## 4. Como carregar uma tabela do DB para um DataFrame Pandas

Este é exemplo de código a ser usado nos notebooks na prática...

In [ ]:
import pandas as pd
import sqlite3



dbfile = "datasets.db"
tabela = "airlines"


db = sqlite3.connect(dbfile)
df = pd.read_sql_query(f'select * from "{tabela}"', db)


df.info()

In [ ]:
df.head(30)